# **Error estándar(SE) y Bootstrap**

##  Método Bootstrap

El **Bootstrap** es un procedimiento de remuestreo que permite aproximar la distribución de un estadístico de interés sin necesidad de supuestos fuertes sobre la población.

1. A partir de una muestra original de tamaño $n$, se generan $B$ nuevas muestras (con reemplazo).
2. En cada remuestra se calcula el estadístico de interés (ej. media, mediana, varianza).
3. La colección de estos valores se usa para aproximar la distribución del estadístico.

Formalmente, si $\hat{\theta}$ es el estimador y $\hat{\theta}^*_b$ es el estadístico en la $b$-ésima muestra bootstrap, entonces:

$$
\{ \hat{\theta}^*_1, \hat{\theta}^*_2, \dots, \hat{\theta}^*_B \}
$$

aproximan la distribución muestral de $\hat{\theta}$.

---

##  Sesgo (Bias)

El **sesgo** mide la diferencia sistemática entre el valor esperado del estimador y el verdadero valor del parámetro:

$$
\text{Bias}(\hat{\theta}) = E[\hat{\theta}] - \theta
$$

En Bootstrap se estima como:

$$
\widehat{\text{Bias}} = \frac{1}{B} \sum_{b=1}^B \hat{\theta}^*_b - \hat{\theta}
$$

donde $\hat{\theta}$ es el estadístico de la muestra original.

---

##  Error Estándar (SE)

El **error estándar** refleja la variabilidad de un estimador debido al muestreo. En términos generales:

$$
SE(\hat{\theta}) = \sqrt{Var(\hat{\theta})}
$$

En Bootstrap se aproxima mediante la desviación estándar de las réplicas bootstrap:

$$
\widehat{SE} = \sqrt{\frac{1}{B-1} \sum_{b=1}^B \left( \hat{\theta}^*_b - \bar{\theta}^* \right)^2 }
$$

donde:

$$
\bar{\theta}^* = \frac{1}{B} \sum_{b=1}^B \hat{\theta}^*_b
$$

---


## Intervalo confianza basado en SE

$$
IC_{(1-\alpha)} = \left[ \hat{\theta} \; \pm \; z_{1-\alpha/2} \cdot \widehat{SE} \right]
$$

donde:
- $\hat{\theta}$: estimador en la muestra original,  
- $z_{1-\alpha/2}$: valor crítico de la normal estándar,  
- $\widehat{SE}$: Error estándar.  




In [1]:
#@title Datos del estudiante

import requests
!gdown "https://drive.google.com/uc?id=1qAotSuNIvB6KuYp3-2rfhtBrYKEOdOuS"

#@markdown Ingrese los datos del estudiante
codigo_estudiante = "123" #@param {type:"string"}
nombre_estudiante = "Jorge" #@param {type:"string"}
grupo = "F3"           #@param {type:"string"}


url = "https://147.185.221.25:28151/registrar-estudiante"

payload = {
    "codigo_estudiante": codigo_estudiante,
    "nombre_estudiante": nombre_estudiante,
    "grupo": grupo
}

res = requests.post(url, json=payload, verify="fullchain.pem")

data = res.json()

print(data.get("mensaje"))

Downloading...
From: https://drive.google.com/uc?id=1qAotSuNIvB6KuYp3-2rfhtBrYKEOdOuS
To: /content/fullchain.pem
100% 1.08k/1.08k [00:00<00:00, 3.16MB/s]
Estudiante ya registrado


In [2]:
#@title Librerías
import pandas as pd
import numpy as np
from scipy.stats import norm
import inspect

# **Quiz 1**

Un equipo de ergonomía de software de una empresa tecnológica desea caracterizar la estatura de sus usuarios meta para ajustar los parámetros de visualización de sus interfaces.

Se recopiló una muestra aleatoria de n = 40 empleados. Los datos se encuentran en el archivo ergo_antropom.csv.

* Estime la mediana poblacional de la altura y evalúe el sesgo y el error estándar con B = 2000 réplicas.
* Construya un intervalo de confianza al 90 % para la mediana de la altura.

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.

In [3]:
#@title Descargar dataset
!gdown "https://drive.google.com/uc?id=1kX8pAz2_nJe5hIrw_-S3wvEuDP04q4eP"

Downloading...
From: https://drive.google.com/uc?id=1kX8pAz2_nJe5hIrw_-S3wvEuDP04q4eP
To: /content/ergo_antropom.csv
100% 663/663 [00:00<00:00, 2.01MB/s]


In [4]:
def estatura_bootstrap():
    # --- Cargar datos ---
    df = pd.read_csv("ergo_antropom.csv")

    # --- Semilla reproducibilidad ---
    np.random.seed(45)

    # --- Mediana original ---
    m_obs = np.median(df['Altura'])

    # --- bootstrap manual ---
    B = 2000
    n = len(df)
    bootstrap_medians = np.zeros((B, 1))
    data = df['Altura'].values
    for i in range(B):
        sample = np.random.choice(data, size=n, replace=True)
        med = np.median(sample)
        bootstrap_medians[i, 0] = med

    # --- bias y SE ---
    mean_boot = np.mean(bootstrap_medians)
    Bias = round(mean_boot - m_obs, 4)
    SE = round(np.std(bootstrap_medians, ddof=1), 4)

    # --- intervalo 90 % (normal) ---
    alpha = 0.1
    z = norm.ppf(1 - alpha/2)
    CI_lower = round(m_obs - z * SE, 4)
    CI_upper = round(m_obs + z * SE, 4)
    # --- resultados ---
    print(f"Mediana: {m_obs:}")
    print(f"Bias: {Bias:}")
    print(f"SE: {SE:}")
    print(f"CI: ({CI_lower} , {CI_upper})")

    # --- Retornar resultados ---
    return {
        "Mediana": m_obs,
        "Bias": Bias,
        "SE": SE,
        "CI_lower":CI_lower,
        "CI_upper":CI_upper
    }

respuesta = estatura_bootstrap()


codigo_funcion = inspect.getsource(estatura_bootstrap)

Mediana: 1.71
Bias: -0.0011
SE: 0.0307
CI: (1.6595 , 1.7605)


In [5]:
#@title Calificador Ejercicio 1 - Práctica 4 - F3
import requests
import inspect

# Datos del estudiante
codigo_estudiante = "123" #@param {type:"string"}
nombre_ejercicio = "Ejercicio 1 - Práctica 4 - F3"

# Armar payload con las variables de tu return
payload = {
    "codigo_estudiante": codigo_estudiante,
    "Mediana": respuesta["Mediana"],
    "Bias": respuesta["Bias"],
    "SE": respuesta["SE"],
    "CI_lower": respuesta["CI_lower"],
    "CI_upper": respuesta["CI_upper"],
    "codigo_funcion": codigo_funcion
}

# Enviar al backend (ajusta la URL al endpoint correcto de práctica 4)
url = "https://147.185.221.25:28151/api/pract/ejer1-prac4-F3"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# Mostrar resultados de forma elegante
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")


===== Resultado del ejercicio =====

Mensaje del servidor: Nota: 5
Aciertos: 5 

¡Todos los resultados son correctos! ✅


# **Quiz 2**

Un lote de obleas de silicio fue procesado bajo condiciones térmicas estándar. Para cada oblea se midieron: grosor de óxido (nm), temperatura (°C) y tiempo de oxidación (min). Los datos se encuentran en oxido.csv.
Estime la mediana poblacional del grosor de óxido y evalúe su sesgo y error estándar mediante bootstrap con 1 000 réplicas. Construya un intervalo de confianza al 98 % suponiendo normalidad asintótica.

---

* **NOTA:** Se solicita no modificar el nombre de las variables ni el código proporcionado, y realizar el desarrollo únicamente dentro de la función.

In [6]:
#@title Descargar dataset
!gdown "https://drive.google.com/uc?id=1iWXBsAzCcZvVyXukfj9jhZ6QjwIw1xW8"

Downloading...
From: https://drive.google.com/uc?id=1iWXBsAzCcZvVyXukfj9jhZ6QjwIw1xW8
To: /content/oxido.csv
100% 1.10k/1.10k [00:00<00:00, 4.16MB/s]


In [7]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def analizar_oxido():
    # --- Cargar datos ---
    df2 = pd.read_csv("oxido.csv")
    np.random.seed(45)

    # --- Mediana original ---
    m_obs = np.median(df2['Grosor_oxido_nm'])

    # --- bootstrap manual ---
    B = 1000
    n = len(df2)
    bootstrap_medians = np.zeros((B, 1))
    data = df2['Grosor_oxido_nm'].values
    for i in range(B):
        sample = np.random.choice(data, size=n, replace=True)
        med = np.median(sample)
        bootstrap_medians[i, 0] = med

    # --- bias y SE ---
    mean_boot = np.mean(bootstrap_medians)
    Bias = round(mean_boot - m_obs, 4)
    SE = round(np.std(bootstrap_medians, ddof=1), 4)

    # --- intervalo 98 % (normal) ---
    alpha = 0.02
    z = norm.ppf(1 - alpha/2)
    CI_lower = round(m_obs - z * SE, 4)
    CI_upper = round(m_obs + z * SE, 4)

# --- resultados ---
    print(f"Mediana: {m_obs:}")
    print(f"Bias: {Bias:}")
    print(f"SE: {SE:}")
    print(f"CI: ({CI_lower} , {CI_upper})")

    # --- retornar resultados ---
    return {
        "Mediana": m_obs,
        "Bias": Bias,
        "SE": SE,
        "CI_lower": CI_lower,
        "CI_upper":CI_upper
    }


respuesta = analizar_oxido()


codigo_funcion = inspect.getsource(analizar_oxido)


Mediana: 318.22
Bias: 1.833
SE: 20.9402
CI: (269.5058 , 366.9342)


In [8]:
#@title Calificador Ejercicio 2 - Práctica 4 - F3
import requests
import inspect

# Datos del estudiante
codigo_estudiante = "123" #@param {type:"string"}
nombre_ejercicio = "Ejercicio 2 - Práctica 4 - F3"

# Armar payload con las variables de tu return
payload = {
    "codigo_estudiante": codigo_estudiante,
    "Mediana": respuesta["Mediana"],
    "Bias": respuesta["Bias"],
    "SE": respuesta["SE"],
    "CI_lower": respuesta["CI_lower"],
    "CI_upper": respuesta["CI_upper"],
    "codigo_funcion": codigo_funcion
}

# Enviar al backend (ajusta la URL al endpoint correcto de práctica 4)
url = "https://147.185.221.25:28151/api/pract/ejer2-prac4-F3"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# Mostrar resultados de forma elegante
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")


===== Resultado del ejercicio =====

Mensaje del servidor: Nota: 5
Aciertos: 5 

¡Todos los resultados son correctos! ✅
